In [1]:
import itertools
from collections import defaultdict
from tqdm import tqdm
import _pickle as cPickle
import os

que_split_path_dict = {

#     "minval":  "../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_minval2014_questions.pkl",
    "val":  "../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_val2014_questions.pkl",
    "train": "../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_train2014_questions.pkl",
}

answer_paths = [
#     "../../datasets/VQA/back-translate/org_bt_minval_target.pkl",
    "../../datasets/VQA/back-translate/org2_bt_val_target.pkl",
    "../../datasets/VQA/back-translate/org2_bt_train_target.pkl",
    ]


ans2label_path = os.path.join("../../datasets/VQA/", "cache", "trainval_ans2label.pkl")
label2ans_path = os.path.join("../../datasets/VQA/", "cache", "trainval_label2ans.pkl")

ans2label = cPickle.load(open(ans2label_path, "rb"))
label2ans = cPickle.load(open(label2ans_path, "rb"))
answer_dict = {}
# use later
answers_data = []

for path in (answer_paths):
    answers = cPickle.load(open(path, "rb"))
    answers_data.append(answers)
    answers = list(itertools.chain.from_iterable(answers))
    for ans in answers:
        answer_dict[ans["question_id"]] = ans["labels"]

print("Read Answers")

negs_path = "../../datasets/VQA/back-translate/train_val_question_negs.pkl"
negs_data = cPickle.load(open(negs_path, "rb"))
negs_dict = {}
for qid, sim_scores, sim_qids in zip(negs_data["qids"], negs_data["sim_scores"], negs_data["sim_qids"]):
    negs_dict[qid] = (sim_scores, sim_qids)

print("Read Matrix")

# create dicts
image_dict = defaultdict(list)
questions_rephrasings = defaultdict(list)
question_dict = {}

# use later
questions_data = []
qids_dict = {}
for split, que_path in que_split_path_dict.items():
    data = cPickle.load(open(que_path, "rb"))
    questions_data.append(data)
    questions_list = data["questions"]
    _dict = {}
    
    # add "rephrasing_of" key
    for _questions in questions_list:
        # only keep the min-qid in same-image ids
        min_qid = min([x['question_id'] for x in _questions])
        assert len(set([x['image_id'] for x in _questions])) == 1
        image_dict[_questions[0]["image_id"]].append(min_qid)
        for _que in _questions:
            question_dict[_que["question_id"]] = _que["question"]
            _dict[_que["question_id"]] = None
    qids_dict[split] = _dict

print("Read Questions")



Read Answers
Read Matrix
Read Questions


In [2]:
save_negs_dict_train = {}
save_negs_dict_val = {}
save_negs_dict_trainval = {}

def filter_negatives(sample):
    # filter same-image questions
    save_dict = {}
    same_image_ids = image_dict[sample["image_id"]]
    fil_same_image_ids = []
    ref_answers = answer_dict[sample["question_id"]]
    for qid in same_image_ids:
        if qid == sample["question_id"]:
            continue
        cand_answers = answer_dict[qid]
        if len(set(ref_answers).intersection(set(cand_answers))) == 0:
            fil_same_image_ids.append(qid)
    save_dict["same_image_questions_neg"] = fil_same_image_ids

    # filter top-k questions
    if sample["question_id"] not in negs_dict:
        return True

    top_k_sim_scores, top_k_questions = negs_dict[sample["question_id"]]
    fil_top_k_questions = []
    fil_top_k_questions_train = []
    fil_top_k_questions_val = []
    
    for qid in top_k_questions:
        cand_answers = answer_dict[qid]
        if len(set(ref_answers).intersection(set(cand_answers))) == 0:
            fil_top_k_questions.append(qid)
            if qid in qids_dict["train"]:
                fil_top_k_questions_train.append(qid)
            elif qid in qids_dict["val"]:
                fil_top_k_questions_val.append(qid)
            else:
                import pdb
                pdb.set_trace()
    
    save_dict["top_k_questions_neg"] = fil_top_k_questions
    save_dict["top_k_questions_neg_train"] = fil_top_k_questions_train
    save_dict["top_k_questions_neg_val"] = fil_top_k_questions_val

    if sample["question_id"] in qids_dict["train"]:
        del save_dict["top_k_questions_neg_val"]
        save_negs_dict_trainval[sample["question_id"]] = save_dict
        save_negs_dict_train[sample["question_id"]] = save_dict
    elif sample["question_id"] in qids_dict["val"]:
        del save_dict["top_k_questions_neg_train"]
        save_negs_dict_trainval[sample["question_id"]] = save_dict
        save_negs_dict_val[sample["question_id"]] = save_dict
    else:
        import pdb
        pdb.set_trace() 
    return False

In [12]:
for que_data, ans_data, que_path, ans_path in \
zip(questions_data[1:], answers_data[1:], list(que_split_path_dict.values())[1:], answer_paths[1:]):
    data = que_data
    answers = ans_data

    questions_list = data["questions"]

    # add "rephrasing_of" key
    for _questions in questions_list:
        rep_id = min([s['question_id'] for s in _questions])
        for _que in _questions:
            _que["rephrasing_of"] = rep_id

    assert len(questions_list) == len(answers)

    # remove questions w/o negatives
    for idx in tqdm(range(len(questions_list)), total=len(questions_list)):
        _updated_ques = []
        _updated_answers = []
        _questions = questions_list[idx]
        _answers = answers[idx]

        for _que, _ans in zip(_questions, _answers):
            delete = filter_negatives(_que)
            if not delete:
                _updated_ques.append(_que)
                _updated_answers.append(_ans)
        questions_list[idx] = _updated_ques
        answers[idx] = _updated_answers

    cPickle.dump(data, open(que_path, "wb"), protocol=2)
    cPickle.dump(answers, open(ans_path, "wb"), protocol=2)
    print(f"Dumped: {que_path}")
    print(f"Dumped: {ans_path}")
    



  2%|▏         | 9197/443757 [00:29<26:16, 275.64it/s]


  4%|▍         | 18389/443757 [00:58<23:40, 299.48it/s]


  6%|▌         | 27455/443757 [01:28<21:55, 316.51it/s]


  8%|▊         | 36495/443757 [01:57<21:03, 322.36it/s]


 10%|█         | 45559/443757 [02:26<21:43, 305.38it/s]


...

 95%|█████████▍| 421565/443757 [22:59<01:10, 313.19it/s]


 97%|█████████▋| 430450/443757 [23:28<00:42, 316.00it/s]


 99%|█████████▉| 439391/443757 [23:56<00:13, 334.13it/s]


100%|██████████| 443757/443757 [24:11<00:00, 305.73it/s]

Dumped: ../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_train2014_questions.pkl
Dumped: ../../datasets/VQA/back-translate/org2_bt_train_target.pkl


In [11]:
keys = list( save_negs_dict_val.keys())
type(save_negs_dict_val[keys[0]]["top_k_questions_neg"][0])

numpy.int64

In [13]:
save_negs_paths = [
"../../datasets/VQA/back-translate/fil_val_question_negs.pkl",
"../../datasets/VQA/back-translate/fil_train_question_negs.pkl",
"../../datasets/VQA/back-translate/fil_trainval_question_negs.pkl",
]

In [14]:
values = list(save_negs_dict_val.values())
lens = [len(x["top_k_questions_neg"]) for x in values]
max(lens)

300

In [15]:
import numpy as np

question_ids = list(save_negs_dict_val.keys())
question_values = list(save_negs_dict_val.values())
question_negs = np.zeros((len(question_ids), 300), dtype=np.int64) + -1
same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int64) + -1

for idx in tqdm(range(len(question_negs))):
    neg_ids = question_values[idx]["top_k_questions_neg_val"]
    same_image_ids = question_values[idx]["same_image_questions_neg"]
    question_negs[idx][:len(neg_ids)] = neg_ids
    try:
        same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
    except:
        import pdb
        pdb.set_trace()


100%|██████████| 1024112/1024112 [00:12<00:00, 81883.75it/s]

In [19]:
cPickle.dump({"qids": question_ids, 
                    "question_negs": question_negs, 
                    "same_image_questions_neg": same_image_questions_neg}, 
             open(save_negs_paths[0], "wb"))


In [22]:
# data = cPickle.load(open(save_negs_paths[0], "rb"))
# data["question_negs"][0]

In [23]:
import numpy as np

question_ids = list(save_negs_dict_train.keys())
question_values = list(save_negs_dict_train.values())
question_negs = np.zeros((len(question_ids), 300), dtype=np.int64) + -1
same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int64) + -1

for idx in tqdm(range(len(question_negs))):
    neg_ids = question_values[idx]["top_k_questions_neg_train"]
    same_image_ids = question_values[idx]["same_image_questions_neg"]
    question_negs[idx][:len(neg_ids)] = neg_ids
    try:
        same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
    except:
        import pdb
        pdb.set_trace()


 68%|██████▊   | 1441538/2119034 [00:26<00:11, 56593.60it/s]


100%|██████████| 2119034/2119034 [00:38<00:00, 54712.76it/s]

In [30]:
cPickle.dump({"qids": question_ids, 
                    "question_negs": question_negs, 
                    "same_image_questions_neg": same_image_questions_neg}, 
             open(save_negs_paths[1], "wb"), protocol=4)
print(save_negs_paths[1])

../../datasets/VQA/back-translate/fil_train_question_negs.pkl


In [32]:
# d = cPickle.load(open(save_negs_paths[1], "rb"))
len(d["qids"])

2119034

In [34]:
import numpy as np

question_ids = list(save_negs_dict_trainval.keys())
question_values = list(save_negs_dict_trainval.values())
question_negs = np.zeros((len(question_ids), 300), dtype=np.int64) + -1
same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int64) + -1

for idx in tqdm(range(len(question_negs))):
    neg_ids = question_values[idx]["top_k_questions_neg"]
    same_image_ids = question_values[idx]["same_image_questions_neg"]
    question_negs[idx][:len(neg_ids)] = neg_ids
    try:
        same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
    except:
        import pdb
        pdb.set_trace()


 18%|█▊        | 557110/3143146 [00:13<01:05, 39447.40it/s]


 35%|███▍      | 1094108/3143146 [00:27<00:52, 39253.49it/s]


 53%|█████▎    | 1662053/3143146 [00:40<00:32, 45084.51it/s]


 72%|███████▏  | 2247472/3143146 [00:54<00:19, 45659.62it/s]


 90%|█████████ | 2839262/3143146 [01:07<00:06, 43600.21it/s]


100%|██████████| 3143146/3143146 [01:14<00:00, 42151.89it/s]

In [35]:
cPickle.dump({"qids": question_ids, 
                    "question_negs": question_negs, 
                    "same_image_questions_neg": same_image_questions_neg}, 
             open(save_negs_paths[2], "wb"), protocol=4)
print(save_negs_paths[2])

../../datasets/VQA/back-translate/fil_trainval_question_negs.pkl


In [ ]:
d = cPickle.load(open(save_negs_paths[2], "rb"))

In [ ]:
        split_path_dict = {
            "train_aug": ["datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_train2014_questions.pkl",
                         "datasets/VQA/back-translate/org3_bt_v2_OpenEnded_mscoco_train2014_questions.pkl", "train"],
            "val_aug": ["datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_val2014_questions.pkl",
                         "datasets/VQA/back-translate/org3_bt_v2_OpenEnded_mscoco_val2014_questions.pkl", "val"],
            "trainval_aug": ["datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_trainval2014_questions.pkl",
                         "datasets/VQA/back-translate/org3_bt_v2_OpenEnded_mscoco_trainval2014_questions.pkl", "trainval"],
            "minval_aug": ["datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_minval2014_questions.pkl",
                         "datasets/VQA/back-translate/org3_bt_v2_OpenEnded_mscoco_minval2014_questions.pkl", "trainval"],
        }


In [ ]:
import pickle as p
import os
os.chdir('/nethome/ykant3/vilbert-multi-task/')

for key in split_path_dict:
    save_path = split_path_dict[key][1]
    questions = p.load(open(split_path_dict[key][0], "rb"))["questions"]
    for que_list in questions:
        for que in que_list:
            del que["languages"]
    
    import pdb
    pdb.set_trace()
    
    p.dump(questions, open(save_path, "wb"))
    